<a href="https://colab.research.google.com/github/rvkgovind/watermarking_effects_on_classification/blob/master/mobilenet_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the packages
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import GlobalAveragePooling2D,Flatten,Input,Activation,BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import time
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
from zipfile import ZipFile
import matplotlib.image as mpimg
import cv2
from skimage.transform import resize
from keras.preprocessing import image
from keras import regularizers,initializers

Using TensorFlow backend.


In [0]:
def get_model():
  base_model = MobileNet(include_top=False, weights='imagenet',input_shape=(224,224,3))

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x=Dropout(0.3)(x)
  x = Dense(512,use_bias=False,kernel_initializer='glorot_uniform')(x)
  x=BatchNormalization()(x)
  x=Activation('relu')(x)
  x=Dropout(0.3)(x)
  predictions = Dense(10, kernel_initializer='uniform',activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions) 
  return model




In [3]:
model=get_model()
for i,layer in enumerate(model.layers):
  print(i,layer.name)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 c

In [0]:
def resize_dataset(X):
  # img_arr is of shape (n, h, w, c)
    x_resized_list = []
    for i in range(X.shape[0]):
        img = X[0]
        resized_img = resize(img, (224, 224))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)

In [0]:
def normalization(X,x):
    X=X.astype('float32')
    x=x.astype('float32')
    mean = np.mean(X,axis=(0,1,2,3))
    std = np.std(x, axis=(0, 1, 2, 3))
    X = (X-mean)/std+1e-7
    x = (x-mean)/std+1e-7
    return X,x,mean,std



#### np.mean()
axis :
If this is a tuple of ints, a mean is performed over multiple axes, instead of a single axis or all the axes as before.

In [0]:
from google.colab.patches import cv2_imshow
def normalize_pred(X):
  Mean = 120.70
  Std = 64.06
  return (X-Mean)/((Std+1e-7))


def extract_zip(path):
  with ZipFile(path, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
  print("Extracted images to the root dir")

def vizualize_data(X_train):
  #print("shape of X_train:",X_train.shape)
  #print("shape of Y_train:",Y_train.shape)
  #print("shape of X_test:",X_test.shape)
  #print("shape of Y_test:",X_test.shape)
  fig=plt.figure()
  for i in range(1,11):
    a=fig.add_subplot(2,5,i)
    imgplot=plt.imshow(X_train[i-1])

def prep_testdata(path):
  X_val=np.zeros([100,224,224,3])
  Y_val=np.zeros([100],dtype='uint8')
  i=0
  test_dictionary={"aeroplane":0,"automobile":1,'bird':2,'cat':3,'deer':4,'dog':5,'frog':6,'horse':7,'ship':8,'truck':9}
  for file in os.listdir(path):
    for key,value in test_dictionary.items():
      if file==key:
        file_path=os.path.join(path,file)
        for images in os.listdir(file_path):
          img_path=os.path.join(file_path,images)
          img = image.load_img(img_path, target_size=(224, 224))
          x = image.img_to_array(img)
          x=np.asarray(x,dtype='uint8')  
          #cv2_imshow(img)
          X_val[i]=x
          Y_val[i]=value
          #print(key)
          #print(value)
          i+=1
  return X_val,Y_val,test_dictionary
def gen_batches_test(X_test,Y_test,BS):
  #infinite loop
  while True:
    X_resized=[]
    Y_labels=[]
    i=0
    while len(X_resized)<BS:
      X = X_train[i]
      X= resize(X, (224, 224))
      X_resized.append(X)
      Y_labels.append(Y_train[i])
      #print(np.array(X_resized).shape)
      #print(np.array(Y_labels).shape)
      i+=1
      
    yield (np.array(X_resized),np.array(Y_labels))

def gen_batches_train(X_train,Y_train,BS,aug=None):
  #infinite loop
  while True:
    X_resized=[]
    Y_labels=[]
    i=0
    while len(X_resized)<BS:
      X = X_train[i]
      X= resize(X, (224, 224))
      X_resized.append(X)
      Y_labels.append(Y_train[i])
      #print(np.array(X_resized).shape)
      #print(np.array(Y_labels).shape)
      i+=1
      # if the data augmentation object is not None, apply it
    if aug is not None:
        (X_resized, Y_labels) = next(aug.flow(np.array(X_resized),Y_labels, batch_size=BS))
      
    yield (np.array(X_resized),np.array(Y_labels))
    

In [0]:
def train_model(model,X_train,Y_train,X_test,Y_test):
  batch_size=256
  epochs=10
  #load the dataset and spilt to train and validate
  #normalize the dataset to mean 0 and variance 1
  for layer in model.layers[:86]:
    layer.trainable=False
  for layer in model.layers[86:]:
    layer.trainable=True
  
  X_train = preprocess_input(X_train)
  X_test=preprocess_input(X_test)
  #conver the targets to one hot vectors using keras.utils.to_categorical
  #data augmentation
  aug = ImageDataGenerator(zca_whitening=True,horizontal_flip=True)
  
  
  # compute quantities required for featurewise normalization
  # (std, mean, and principal components if ZCA whitening is applied)
  datagen=gen_batches_train(X_train,Y_train,batch_size,aug=None)
  testdata_gen=gen_batches_test(X_test,Y_test,batch_size)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy']) 
  filepath="weights.best.hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
  callbacks_list = [checkpoint]
  # fits the model on batches with real-time data augmentation:
  history=model.fit_generator(datagen,steps_per_epoch=X_train.shape[0]// batch_size, epochs=epochs,
                              validation_data=testdata_gen,validation_steps=X_test.shape[0]//batch_size,callbacks=callbacks_list)
  
  #compile the model by mentioning the loss function,optimizer, accuracy to measure
 
  return model 

In [8]:
if __name__=='__main__':
  start=time.time()
  #[X_train, Y_train,X_test, Y_test]= load_cifar10_data(img_rows,img_cols)
  (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
  Y_train = keras.utils.to_categorical(Y_train, 10)
  Y_test = keras.utils.to_categorical(Y_test, 10)
  if os.path.exists('./weights.best.hdf5'):
    model=get_model()
    extract_zip('test.zip')
    test_lab={}
    [X_cross,Y_cross,test_lab]=prep_testdata('./test')
    #vizualize_data(X)
    #X_norm=normalize_pred(X)
    Y=keras.utils.to_categorical(Y_cross,10)
    model.load_weights('./weights.best.hdf5')
    i=0
    k=list(test_lab.keys())
    v=list(test_lab.values())
    residual=[] 
    for i in range(0,100):
      imag = X_cross[i][:, :, (2, 1, 0)]
      cv2_imshow(imag)
      x=np.expand_dims(X_cross[i], axis=0)
      x=preprocess_input(x)
      predict=model.predict(x)
      residual.append(np.argmax(predict)!=np.argmax(Y[i]))
      prediction=np.argmax(predict)
      true_label=np.argmax(Y[i])
      print('prediction:',k[v.index(prediction)])
      print('true label:',k[v.index(true_label)])
      i+=1
      
    loss = sum(residual)/len(residual)
    print('loss during testing with test images:',loss)
    
  else:
    model=get_model()
    trained_model=train_model(model,X_train,Y_train,X_test,Y_test)
  print("time taken for execution:",time.time()-start)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:334: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


195/195 [==============================] - 426s 2s/step - loss: 0.1115 - acc: 0.9781 - val_loss: 0.5998 - val_acc: 0.8242

Epoch 00001: val_acc improved from -inf to 0.82422, saving model to weights.best.hdf5
Epoch 2/10
195/195 [==============================] - 397s 2s/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.5831 - val_acc: 0.8164

Epoch 00002: val_acc did not improve from 0.82422
Epoch 3/10
195/195 [==============================] - 401s 2s/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.5594 - val_acc: 0.8359

Epoch 00003: val_acc improved from 0.82422 to 0.83594, saving model to weights.best.hdf5
Epoch 4/10
195/195 [==============================] - 403s 2s/step - loss: 6.3368e-04 - acc: 1.0000 - val_loss: 0.5774 - val_acc: 0.8242

Epoch 00004: val_acc did not improve from 0.83594
Epoch 5/10
195/195 [==============================] - 402s 2s/step - loss: 3.9731e-04 - acc: 1.0000 - val_loss: 0.5686 - val_acc: 0.8164

Epoch 00005: val_acc did not improve from 0.83594
Epoch 6/